In [ ]:
# 데이터 구조
import pandas as pd
# 데이터 시각화
import matplotlib.pyplot as plt
import matplotlib
# 데이터 scaling
from sklearn.preprocessing import StandardScaler
# 데이터 분할:train, test
from sklearn.model_selection import train_test_split
# 분류 SVM
from sklearn.svm import SVC
# 최적 모델, 파라미터 탐색
from sklearn.model_selection import GridSearchCV
# 모델 성능 평가
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [ ]:
df_raw = pd.read_csv("../data/bank.csv")
df_raw.head()

In [ ]:
# 결측치 확인
df_raw.isnull().sum()
# JOB 변수의 결측치는 "Other"로 대체
df_raw["JOB"].fillna("Other", inplace = True)
# 숫자형 변수의 결측치는 해당 변수의 평균값으로 대체
df_raw.fillna(df_raw.mean(), inplace=True)

In [ ]:
# get_dummies: 데이터의 문자형 변수에 대한 더미변수 생성
df_raw_dummy = pd.get_dummies(df_raw)
# 더미변수 생성된 데이터의 상위 5개 row를 확인
df_raw_dummy.head()

In [ ]:
# 학습용/평가용 데이터 분리
df_train, df_test = train_test_split(df_raw_dummy, test_size = 0.3, random_state = 1234)
print("학습용 데이터의 크기 : {}".format(df_train.shape))
print("평가용 데이터의 크기 : {}".format(df_test.shape))

In [ ]:
# 설명변수와 목표변수를 분리
df_train_y = df_train["BAD"]
df_train_x = df_train.drop("BAD", axis = 1, inplace = False)
df_test_y = df_test["BAD"]
df_test_x = df_test.drop("BAD", axis = 1, inplace = False)

In [ ]:
svm_uncustomized = SVC(random_state=1234)
svm_uncustomized.fit(df_train_x, df_train_y)
# 학습용 데이터 셋 정확도
print("학습용 데이터의 정확도 : {:.3f}".format(svm_uncustomized.score(df_train_x, df_train_y)))
# 평가용 데이터 셋 정확도
print("평가용 데이터의 정확도 : {:.3f}".format(svm_uncustomized.score(df_test_x, df_test_y)))

In [ ]:
# 변수명 저장
v_feature_name = df_train_x.columns
# StandardScaler 적용
scaler = StandardScaler()
df_train_scaled_x = scaler.fit_transform(df_train_x)
df_test_scaled_x = scaler.transform(df_test_x)
df_train_scaled_x = pd.DataFrame(df_train_scaled_x, columns=v_feature_name)
df_test_scaled_x = pd.DataFrame(df_test_scaled_x, columns=v_feature_name)
df_train_scaled_x.head()

In [ ]:
svm_scaled = SVC(random_state=1234)
svm_scaled.fit(df_train_scaled_x, df_train_y)
# 학습용 데이터 셋 정확도
print("학습용 데이터의 정확도 : {:.3f}".format(svm_scaled.score(df_train_scaled_x, df_train_y)))
# 평가용 데이터 셋 정확도
print("평가용 데이터의 정확도 : {:.3f}".format(svm_scaled.score(df_test_scaled_x, df_test_y)))

In [ ]:
estimator = SVC(random_state=1234)
# 구하고자 하는 parameter와 범위
param_grid = {"C": [1, 10, 100], "gamma": [0.01, 0.1, 1, 100]}
# 정확도가 높은 최적 parameter 찾기
grid_rf = GridSearchCV(estimator, param_grid, scoring="accuracy", n_jobs = -1)
grid_rf.fit(df_train_scaled_x, df_train_y)
print("best estimator model: \n{}".format(grid_rf.best_estimator_))
print("\nbest parameter: \n{}".format(grid_rf.best_params_))
print("\nbest score: \n{}".format(grid_rf.best_score_.round(3)))

In [ ]:
# 최종 모델
svc_final = SVC(gamma = 0.1, C = 10, random_state=1234)
svc_final.fit(df_train_scaled_x, df_train_y)
# 평가
y_pred = svc_final.predict(df_test_scaled_x)
# 학습용 데이터 셋 정확도
print("학습용 데이터의 정확도 : {:.3f}".format(svc_final.score(df_train_scaled_x, df_train_y)))
# 평가용 데이터 셋 정확도
print("평가용 데이터의 정확도 : {:.3f}".format(svc_final.score(df_test_scaled_x, df_test_y)))
# confusion matrix
print("Confusion matrix: \n{}".format(confusion_matrix(df_test_y, y_pred)))
# classification Report
print("Classification Report : ", classification_report(df_test_y, y_pred))